# Στοιχεία Ομάδας

**Αριθμός Ομάδας:** 66

**Ονοματεπώνυμα και ΑΜ:**

Τσιλιβής Θεόδωρος 03116032

Στόικου Θεοδότη 03117085

Ποταμίτου Νεφέλη 03117709

# Imports and stuff


In [ ]:
LOG_DIR = 'tb_log/'
!mkdir -p LOG_DIR
%load_ext tensorboard
# %tensorboard --logdir {LOG_DIR}
!pip install --upgrade pip
!pip install --upgrade stable_baselines3[extra]
# we need a specific version of gym because of this issue: https://github.com/DLR-RM/stable-baselines3/issues/294
!pip install gym==0.17.3
atari_env_name='Berzerk-v0'
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
import datetime # For filenames while logging
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import A2C
from stable_baselines3 import PPO


In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  -y # For visualization

In [ ]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
video_folder = '/videos'

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(eval_env, model, video_length=500, prefix='', video_folder=video_folder):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs, deterministic=True)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

# Παίζοντας Atari με βαθιά ενισχυτική μάθηση

---




## Tensorboard
Ορίζουμε το directory με τα log files

In [ ]:
LOG_DIR = 'tb_log/'
!mkdir -p LOG_DIR



### Colab magic
στο Colab δεν χρειάζεται το ngrok γιατί υπάρχει ειδικό magic για το TensorBoard.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir {LOG_DIR}

```
%load_ext tensorboard
%tensorboard --logdir {LOG_DIR}
```
Σε εμάς δεν λειτούργησε, ωστόσο μας αναφέρθηκε ότι δουλεύει αν τρέξετε την πρώτη γραμμή πριν την εκπαίδευση και την δεύτερη μετά την εκπαίδευση (όταν έχει περιεχόμενο ο φάκελος LOG_DIR).

Παρόλαυτά εμείς θέλουμε να βλέπουμε ζωντανά το TensorBoard κατά την εκπαίδευση και όχι μετά ώστε πχ να μπορούμε να διακόψουμε μοντέλα που δεν έχουν ικανοποιητικές γραφικές. Αυτό θέλουμε να το πετύχουμε χωρίς callbacks
.

Και πάλι προτείνουμε το Kaggle version.


## Εγκατάσταση βιβλιοθήκης και gym

In [ ]:
!pip install --upgrade pip
!pip install --upgrade stable_baselines3[extra]
# we need a specific version of gym because of this issue: https://github.com/DLR-RM/stable-baselines3/issues/294
!pip install gym==0.17.3

## Ορισμός παιχνιδιού

Βάλτε εδώ το string που αντιστοιχεί στο παιχνίδι σας. Για την ονοματολογία των περιβαλλόντων:
- v0 vs v4: v0 has repeat_action_probability of 0.25 (meaning 25% of the time the previous action will be used instead of the new action), while v4 has 0 (always follow your issued action)
- Deterministic: a fixed frameskip of 4, while for the env without Deterministic, frameskip is sampled from [2,4]
- There is also NoFrameskip-v4 with no frame skip and no action repeat stochasticity.

Αναλυτικότερα στην εκφώνηση της άσκησης.

In [ ]:
atari_env_name='Berzerk-v4'

## Δημιουργία περιβάλλοντος

In [ ]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack

In [ ]:
# Με τις συναρτήσεις που ακολουθούν κάνουμε την ίδια προεπεξεργασία με την DeepMind

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

## Εκπαίδευση

Θα εκπαιδεύσουμε ένα δίκτυο deep q-learning (DQN) όπως αυτό της Deepmind. Σημειώστε ότι τα timesteps είναι πολύ λίγα και ότι δεν κάνουμε διερεύνηση στις παραμέτρους του μοντέλου που μπορείτε να βρείτε [εδώ](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html#parameters).

In [ ]:
import datetime # For filenames while logging
from stable_baselines3 import DQN

In [ ]:


model_name='dqn-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [ ]:
max_steps=100
dqn_model.learn(total_timesteps=max_steps)

In [ ]:
%tensorboard --logdir {LOG_DIR}

## Εκτίμηση απόδοσης

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

## Σώσιμο εκπαιδευμένου μοντέλου

In [ ]:
dqn_model.save("dqn_pong")

Το μοντέλο θα αποθηκευθεί ως zip και μπορείτε να το κατεβάσετε τοπικά από το αριστερό sidebar του Colab στο "Files" και μετά στο ellipsis menu πάνω στο filename.

## Φόρτωση εκπαιδευμένου μοντέλου

Από το αριστερό sidebar του Colab και το "Files" ανεβάστε το αρχείο zip του εκπαιδευμένου μοντέλου.

Εδώ θα ανεβάσουμε ένα μοντέλο Α2C που έχουμε εκπαιδεύσει νωρίτερα. Μπορείτε να το κατεβάσετε από [εδώ](https://drive.google.com/uc?export=download&id=1COsaNOH8SjbpxxIYc5lOF-QUiUJGU5ZB). 

Αν χρειαστεί μετονομάστε το αρχείο σε a2c_pong.zip

In [ ]:
from stable_baselines3 import A2C
# !wget --no-check-certificate https://www.dropbox.com/s/zm02848gzbx3jsl/a2c_pong.zip?dl=1 -O a2c_berzerk.zip
# a2c_model = A2C.load("a2c_berzerk.zip", verbose=10)

In [ ]:
model_name='a2c-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model = A2C('MlpPolicy', env, verbose=1, tensorboard_log=model_log)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [ ]:
max_steps=10000
a2c_model.learn(total_timesteps=max_steps)

In [ ]:
%tensorboard --logdir {LOG_DIR}

## Συνέχιση της εκπαίδευσης

In [ ]:
# Προσοχή, το n_envs πρέπει να είναι πάντα το ίδιο (αυτό με το οποίο έγινε η αρχική εκπαίδευση του μοντέλου)
new_env = make_atari_env(atari_env_name, n_envs=4, seed=0)
# Frame-stacking with 4 frames
new_env = VecFrameStack(new_env, n_stack=4)
# Το μοντέλο δεν έχει μαζί του το περιβάλλον, πρέπει να του το αναθέσουμε ξανά.
a2c_model.set_env(new_env)

Wrapping the env in a VecTransposeImage.


In [ ]:
#max_steps=10000
#a2c_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 365.0 (+/-122.57650672131263)


In [ ]:
a2c_model.save("a2c_pong")

## Καταγραφή video του actual gameplay του πράκτορα

Ωραία τα διαγράμματα, ωραίο το TensorBoard, αλλά θέλουμε οπωσδήποτε να δούμε τον πράκτορα να παίζει το παιχνίδι!

In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  -y # For visualization

In [ ]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
video_folder = '/videos'

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(eval_env, model, video_length=500, prefix='', video_folder=video_folder):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs, deterministic=True)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [ ]:
# record_video(test_env, dqn_model, video_length=5000, prefix='dqn_pong')

In [ ]:
record_video(test_env, a2c_model, video_length=5000, prefix='a2c_pong')

In [ ]:
# show_videos(video_path = video_folder, prefix='dqn')

In [ ]:
show_videos(video_path = video_folder, prefix='a2c')

Μπορείτε να κατεβάσετε το video με hover πάνω του, κάτω δεξιά στο ellipsis menu και "Download".

# Structured approach


## DQN Deterministic-v4

In [ ]:
# -v4, Deterministic
atari_env_name='BerzerkDeterministic-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

In [ ]:
model_name='dqn-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

In [ ]:
dqn_model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000, exploration_final_eps=0.15)
max_steps=10000
dqn_model.learn(total_timesteps=max_steps)


In [ ]:
%tensorboard --logdir {LOG_DIR}

In [ ]:
from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
callback = StopTrainingOnMaxEpisodes
mean_reward, std_reward = evaluate_policy(dqn_model, test_env, callback=callback, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 115.0 (+/-22.9128784747792)


In [ ]:
record_video(test_env, dqn_model, video_length=5000, prefix='dqn_BerzerkDeterministic-v4')

In [ ]:
show_videos(video_path = video_folder, prefix='dqn')

In [ ]:
dqn_model.save("dqn_BerzerkDeterministic-v4")
obs = env.reset()

##DQN-v4 (simple)

In [ ]:
atari_env_name='Berzerk-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

model_name='dqn-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000, exploration_final_eps=0.15)
max_steps=10000
dqn_model.learn(total_timesteps=max_steps)

In [ ]:
%tensorboard --logdir {LOG_DIR}

In [ ]:
from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
callback = StopTrainingOnMaxEpisodes
mean_reward, std_reward = evaluate_policy(dqn_model, test_env, callback=callback, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
record_video(test_env, dqn_model, video_length=5000, prefix='dqn_Berzerk-v4')

In [ ]:
show_videos(video_path = video_folder, prefix='dqn')

In [ ]:
dqn_model.save("dqn_Berzerk-v4")
obs = env.reset()

##DQN NoFrameskip-v4

In [ ]:
atari_env_name='BerzerkNoFrameskip-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

model_name='dqn-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model = DQN('CnnPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000, exploration_final_eps=0.15)
max_steps= 1000000
dqn_model.learn(total_timesteps=max_steps)

# %tensorboard --logdir {LOG_DIR}

# from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
# callback = StopTrainingOnMaxEpisodes
# mean_reward, std_reward = evaluate_policy(dqn_model, test_env, callback=callback, n_eval_episodes=10)
# print(f"Eval reward: {mean_reward} (+/-{std_reward})")

# record_video(test_env, dqn_model, video_length=5000, prefix='dqn_BerzerkNoFrameskip-v4')

# show_videos(video_path = video_folder, prefix='dqn')

dqn_model.save("dqn_BerzerkNoFrameskip-v4")
obs = env.reset()

In [ ]:
# %tensorboard --logdir {LOG_DIR}

from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
callback = StopTrainingOnMaxEpisodes
mean_reward, std_reward = evaluate_policy(dqn_model, test_env, callback=callback, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

record_video(test_env, dqn_model, video_length=5000, prefix='dqn_BerzerkNoFrameskip-v4')

show_videos(video_path = video_folder, prefix='dqn')

##A2C Deterministic-v4

In [ ]:
atari_env_name='BerzerkDeterministic-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

model_name='a2c-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model = A2C('MlpPolicy', env, verbose=1, tensorboard_log=model_log)
max_steps=10000
a2c_model.learn(total_timesteps=max_steps)

%tensorboard --logdir {LOG_DIR}

from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
callback = StopTrainingOnMaxEpisodes
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, callback=callback, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

record_video(test_env, a2c_model, video_length=5000, prefix='a2c_BerzerkDeterministic-v4')

show_videos(video_path = video_folder, prefix='a2c')

a2c_model.save("a2c_BerzerkDeterministic-v4")
obs = env.reset()

##A2C-v4 (simple)

In [ ]:
atari_env_name='Berzerk-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

model_name='a2c-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model = A2C('MlpPolicy', env, verbose=1, tensorboard_log=model_log)
max_steps=10000
a2c_model.learn(total_timesteps=max_steps)

# %tensorboard --logdir {LOG_DIR}

# from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
# callback = StopTrainingOnMaxEpisodes
# mean_reward, std_reward = evaluate_policy(a2c_model, test_env, callback=callback, n_eval_episodes=10)
# print(f"Eval reward: {mean_reward} (+/-{std_reward})")

# record_video(test_env, a2c_model, video_length=5000, prefix='a2c_Berzerk-v4')

# show_videos(video_path = video_folder, prefix='a2c')

a2c_model.save("a2c_Berzerk-v4")
obs = env.reset()

##A2C NoFrameskip-v4

In [ ]:
atari_env_name='BerzerkNoFrameskip-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

model_name='a2c-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model = A2C('MlpPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000, exploration_final_eps=0.15)
max_steps=10000
a2c_model.learn(total_timesteps=max_steps)

%tensorboard --logdir {LOG_DIR}

from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
callback = StopTrainingOnMaxEpisodes
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, callback=callback, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

record_video(test_env, a2c_model, video_length=5000, prefix='a2c_BerzerkNoFrameskip-v4')

show_videos(video_path = video_folder, prefix='a2c')

a2c_model.save("a2c_BerzerkNoFrameskip-v4")
obs = env.reset()

##PPO Deterministic-v4

In [ ]:
atari_env_name='BerzerkDeterministic-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

model_name='ppo-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

ppo_model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=model_log, )
max_steps=10000
ppo_model.learn(total_timesteps=max_steps)

%tensorboard --logdir {LOG_DIR}

from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
callback = StopTrainingOnMaxEpisodes
mean_reward, std_reward = evaluate_policy(ppo_model, test_env, callback=callback, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

record_video(test_env, ppo_model, video_length=5000, prefix='ppo_BerzerkDeterministic-v4')

show_videos(video_path = video_folder, prefix='ppo')

ppo_model.save("a2c_BerzerkDeterministic-v4")
obs = env.reset()

##PPO-v4 (simple)

In [ ]:
atari_env_name='Berzerk-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

model_name='ppo-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

ppo_model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=model_log)
max_steps=10000
ppo_model.learn(total_timesteps=max_steps)

# %tensorboard --logdir {LOG_DIR}

# from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
# callback = StopTrainingOnMaxEpisodes
# mean_reward, std_reward = evaluate_policy(ppo_model, test_env, callback=callback, n_eval_episodes=10)
# print(f"Eval reward: {mean_reward} (+/-{std_reward})")

# record_video(test_env, ppo_model, video_length=5000, prefix='ppo_Berzerk-v4')

# show_videos(video_path = video_folder, prefix='ppo')

ppo_model.save("ppo_Berzerk-v4")
obs = env.reset()

##PPO NoFrameskip-v4

In [ ]:
atari_env_name='BerzerkNoFrameskip-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

model_name='ppo-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

ppo_model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000, exploration_final_eps=0.15)
max_steps=10000
ppo_model.learn(total_timesteps=max_steps)

%tensorboard --logdir {LOG_DIR}

from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
callback = StopTrainingOnMaxEpisodes
mean_reward, std_reward = evaluate_policy(ppo_model, test_env, callback=callback, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

record_video(test_env, ppo_model, video_length=5000, prefix='ppo_BerzerkNoFrameskip-v4')

show_videos(video_path = video_folder, prefix='ppo')

ppo_model.save("ppo_BerzerkNoFrameskip-v4")
obs = env.reset()

## Random Agent


In [ ]:
import numpy as np
# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
obs = env.reset()
n_steps = 1000
total_reward = 0
episodes = 0
for _ in range(n_steps):
    action = np.array([env.action_space.sample()])
    obs, reward, done, info = env.step(action)
    total_reward += reward[0]
    if done:
      print("Episode:{} Score:{}".format(episodes + 1,total_reward))
      obs = env.reset()
      episodes += 1
    if episodes == 10:
      break
      print("Final Score:{} Number of Episodes:{}".format(total_reward, episodes))

## Βελτιστοποιηση agent A2C

In [ ]:
atari_env_name='Berzerk-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=4, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=4, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

model_name='a2c-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model = A2C('MlpPolicy', env, verbose=1, tensorboard_log=model_log)
max_steps=1000000
a2c_model.learn(total_timesteps=max_steps)

a2c_model.save("a2c_Berzerk-v4")
obs = env.reset()

In [ ]:
%tensorboard --logdir {LOG_DIR}

In [ ]:
from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
callback = StopTrainingOnMaxEpisodes
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, callback=callback, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 610.0 (+/-73.48469228349535)


In [ ]:
record_video(test_env, a2c_model, video_length=5000, prefix='a2c_Berzerk-v4')

show_videos(video_path = video_folder, prefix='a2c')

### Πολύ μικρό learning rate

In [ ]:
atari_env_name='Berzerk-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=4, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=4, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

model_name='a2c-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model = A2C('MlpPolicy', env, verbose=1, tensorboard_log=model_log, learning_rate=0.00001)
max_steps=200000
a2c_model.learn(total_timesteps=max_steps)

a2c_model.save("a2c_Berzerk-v4")
obs = env.reset()

In [ ]:
from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
callback = StopTrainingOnMaxEpisodes
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, callback=callback, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 180.0 (+/-67.82329983125268)


In [ ]:
record_video(test_env, a2c_model, video_length=5000, prefix='a2c_Berzerk-v4')

show_videos(video_path = video_folder, prefix='a2c')

### Μεγάλο gamma coefficient

In [ ]:
atari_env_name='Berzerk-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=4, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=4, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

model_name='a2c-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model = A2C('MlpPolicy', env, verbose=1, tensorboard_log=model_log, gamma = .90, gae_lambda=.9)
max_steps=200000
a2c_model.learn(total_timesteps=max_steps)

a2c_model.save("a2c_Berzerk-v4")
obs = env.reset()

In [ ]:
from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
callback = StopTrainingOnMaxEpisodes
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, callback=callback, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 365.0 (+/-150.08331019803634)


In [ ]:
record_video(test_env, a2c_model, video_length=5000, prefix='a2c_Berzerk-v4')

show_videos(video_path = video_folder, prefix='a2c')

### Μεγάλο learning rate με μικρό gamma coefficient

In [ ]:
atari_env_name='Berzerk-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=4, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=4, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

model_name='a2c-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model = A2C('MlpPolicy', env, verbose=1, tensorboard_log=model_log, gamma = .90, learning_rate=0.003)
max_steps=200000
a2c_model.learn(total_timesteps=max_steps)

a2c_model.save("a2c_Berzerk-v4")
obs = env.reset()

In [ ]:
from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
callback = StopTrainingOnMaxEpisodes
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, callback=callback, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 555.0 (+/-127.37739202856997)


In [ ]:
record_video(test_env, a2c_model, video_length=5000, prefix='a2c_Berzerk-v4')

show_videos(video_path = video_folder, prefix='a2c')

### CNN Policy

In [ ]:
atari_env_name='Berzerk-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing
env = make_atari_env(atari_env_name, n_envs=4, seed=0)
# Frame-stacking with 4 frames. Με 1 frame ο αλγόριθμος ξέρει τη θέση των πραγμάτων, με 2 frames την ταχύτητα, με 3 την επιτάχυνση και με 4 το jerk
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=4, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

model_name='a2c-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model = A2C('CnnPolicy', env, verbose=1, tensorboard_log=model_log)
max_steps=200000
a2c_model.learn(total_timesteps=max_steps)

a2c_model.save("a2c_Berzerk-v4")
obs = env.reset()

In [ ]:
from stable_baselines3.common.callbacks import StopTrainingOnMaxEpisodes
callback = StopTrainingOnMaxEpisodes
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, callback=callback, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 612.0 (+/-122.45815611873306)


In [ ]:
record_video(test_env, a2c_model, video_length=5000, prefix='a2c_Berzerk-v4')

show_videos(video_path = video_folder, prefix='a2c')